# Assignment 9

# Write a Python program based on the Week 9 class exercises that:
I will implements a keyword taxonomy of topic classes with Word2Vec 

In [1]:
#Loads previously obtained dataset of Webhose news articles
import json 
    
#read data from json file
json_data=open("Word2Vec_LSH_duplication.json").readlines()
#print(json_data)

#Output: print the number of articles, titles and publish dates of the first 100 articles.
feeds_read_from_file = []
for line in json_data:
    #print(line)
    feeds_read_from_file.append(json.loads(line))
    
print(len(feeds_read_from_file))

# feed_titles = []

# for feed in feeds_read_from_file:
#     #print(feed)
#     #print(str(feed[1]))
#     if feed[1] != '':
#         feed_titles.append(str(feed[1]))

feed_titles_index = []       
for i in range(len(feeds_read_from_file)):
    if feeds_read_from_file[i][1] != '':
        feed_titles_index.append([i,feeds_read_from_file[i][1]])

print(feed_titles_index)

4868
[[0, 'Two self-paced OpenShift Workshops for Developers'], [1, 'How Finding a Hobby Will Make You A Better Entrepreneur'], [2, 'Cognizant: Continued Strength Across Verticals, With 25% Stock Upside Likely In 2019-20 - Cognizant Technology Solutions Corporation (NASDAQ:CTSH)'], [3, 'Group Puzzles Out Silicon Specs'], [4, 'When AIs go to war: Autonomous cyber weapons ‘inevitable’'], [5, 'Tuesday Night'], [6, 'ROOMZ expands into the UK and Ireland'], [7, 'Digital skills are Africa’s ticket to prosperity'], [8, 'LevaData Announces 2019 Cognitive Sourcing Summit'], [9, 'Exeter Township woman began career in computer programming in 1962'], [10, 'Nutanix appoints new General Manager to oversee UK and Ireland'], [11, 'Why We Love Domain promo Code (And You Should, Too!)'], [12, 'Hedera Hashgraph Opens Public Network'], [13, 'Compute Express Link Consortium (CXL) Officially Incorporates; Announces Expanded Board of Directors'], [14, 'How to Personalize Digital Experiences in Finance (Paul 

## Implements a keyword taxonomy of topic classes.

In [2]:
import gensim, operator
from scipy import spatial
import numpy as np
from gensim.models import KeyedVectors

#model_path = '/Users/javid/projects/enaibl/models/wordvec/'
model_path = '/Users/rissacao/downloads/'

In [3]:
def load_wordvec_model(modelName, modelFile, flagBin):
    print('Loading ' + modelName + ' model...')
    model = KeyedVectors.load_word2vec_format(model_path + modelFile, binary=flagBin)
    print('Finished loading ' + modelName + ' model...')
    return model

model_word2vec = load_wordvec_model('Word2Vec', 'GoogleNews-vectors-negative300.bin.gz', True)
#model_fasttext = load_wordvec_model('FastText', 'fastText_wiki_en.vec', False)

Loading Word2Vec model...
Finished loading Word2Vec model...


In [4]:
# BUILD YOUR OWN TAXONOMY BASED ON LDA and MANUAL DATA EXPLORATION
topic_taxonomy = {
    "leader":
    {
        "leader roles":"executive leadership manager CEO COO CTO CMO director general mananger VP",
        "Leadership Approval":"approved agreed supported aligns",
        "Leadership Rejection":"reject refuse disqualify contradict",
        "Leadership Training":"learn train learning training"
    },
    
     "Tech giant":
    {
        "Microsoft":"MS Microsoft Azure",
        "Amazon":"AWS Amazon",
        "Facebook":"FB Facebook",
        "Google":"Google",
        "IBM":"International Business Machines Corporation IBM"
    },

    "technology areas":
    {
        "AI":"artificial intelligence AI machine learning",
        "IOT":"Internet of things IOT smart home smart city",
        "RPA":"intelligent automation ibm automation robotics process automation virtual workforce",
        "Quantum Computing":"Quantum Computing",
        "Cloud":"cloud vmvare"
    },
   
    "business company":
    {
        "Business Expansion":    "localization globalization to acquire buy acquisition merge invest purchase spend subsidize win secure contract venture",
        "Business Reorganization": "bankruptcy withdraw divest exit pullout sale sell outflow writeoff split",
        "Business Partnership":  "partner team collaboration alliance deal contract",
        "Operational Outage":    "operational outage disruption halt end cancellation delay",
        "Operations Resume":     "restore restart resume maintain operations",
        "Business Statement":    "shareholder issues releases statement results policy denial confirmation",
        "Business Settlement":   "settlement refund moneyback compensate reimburse",
        "Business Competition":  "tops leads to rival competitor entrant competitive advantage"
    },
    
    "Geography location country":
    {
        "Country":  "UK US Ireland China Japan Korea Mexico Colombia",
        "Province": "New York California Utah Texus North Carolina Georgia South California Florida",
        "City": "Chicago New York Las Vegas Los Angeles San Francisco Toronto Vancouver Beijing Shanghai Seoul "
    },
    
    "Business Relationship ":
    {
        "Competitor":    "rival competitor competing with ",
        "Partnership":   "partner Allies team up"
    },
    
    "earnings revenue sales stock profits income":
    {
        "Revenue Growth":  "up post gain increase growth climb rise jump surge soar boost",
        "Revenue Drop": "drop decline down fall lose value decrease stumble plunge slip plummet"
    },
    
    "market expansion":
    {
        "New Technology":   "new technology system development invention patent driverless self-driving automation",
        "New Project":      "new project initiative field discovery exploration",
        "New Product":      "new unveil release announce launch build introduce produced concept model"

    },
    
    "product":
    {
        "Product release":  "create build develop release",
        "Watson": "Assistant Natural lanaguage processing NLP computer vision voice recognition"
        
    },
    
    "sport":
    {
        "Sport": "game sports tournament NBA FIFA"
    },
    
    "outlook prediction" :
    {
        "Positive Outlook": "raising upgrade outperform strong growing positive optimistic high up profitable",
        "Negative Outlook": "overcome shortern cut pessimistic warn tough times decline weak negative down low downgrade fail drop"
    },

}

## Classify Webhose article titles against the developed taxonomy using semantic_text_similarity package 

In [5]:
def vec_similarity(input1, input2, vectors):
    term_vectors = [np.zeros(300), np.zeros(300)]
    terms = [input1, input2]
        
    for index, term in enumerate(terms):
        for i, t in enumerate(term.split(' ')):
            try:
                term_vectors[index] += vectors[t]
            except:
                term_vectors[index] += 0
        
    result = (1 - spatial.distance.cosine(term_vectors[0], term_vectors[1]))
    if result is 'nan':
        result = 0
        
    return result

In [6]:
# function checks whether the input words are present in the vocabulary for the model
def vocab_check(vectors, words):
    
    output = list()
    for word in words:
        if word in vectors.vocab:
            output.append(word.strip())
            
    return output

In [7]:
# function calculates similarity between two strings using a particular word vector model
def calc_similarity(input1, input2, vectors):
    s1words = set(vocab_check(vectors, input1.split()))
    s2words = set(vocab_check(vectors, input2.split()))
    
    output = vectors.n_similarity(s1words, s2words)
    return output

In [8]:
# function takes an input string, runs similarity for each item in topic_taxonomy, sorts and returns top 3 results
def classify_topics(input, vectors):
    feed_score = dict()
    for key, value in topic_taxonomy.items():
        max_value_score = dict()
        for label, keywords in value.items():
            max_value_score[label] = 0
            topic = (key + ' ' + keywords).strip()
            max_value_score[label] += float(calc_similarity(input, topic, vectors))
            
        sorted_max_score = sorted(max_value_score.items(), key=operator.itemgetter(1), reverse=True)[0]
        feed_score[sorted_max_score[0]] = sorted_max_score[1]
    return sorted(feed_score.items(), key=operator.itemgetter(1), reverse=True)[:3]

In [9]:
title_classifier = []
print(len(feed_titles_index))

for i in range(len(feed_titles_index)):
    
    if __name__ == '__main__':
        
        try:
            output= classify_topics(str(feed_titles_index[i][1]), model_word2vec)
            print(output)
            title_classifier.append([feed_titles_index[i][0], output])
            
        except ZeroDivisionError:
            output = 'N/A'
            
#print(title_classifier)

4809
[('IBM', 0.3782532513141632), ('Product release', 0.32253921031951904), ('Partnership', 0.3161008358001709)]
[('Partnership', 0.3844800591468811), ('IOT', 0.318482369184494), ('Province', 0.2783328890800476)]
[('IBM', 0.5760565996170044), ('Partnership', 0.48983243107795715), ('Quantum Computing', 0.4308468699455261)]
[('IBM', 0.457440048456192), ('Quantum Computing', 0.43630194664001465), ('Partnership', 0.35222360491752625)]
[('AI', 0.4435213804244995), ('Leadership Training', 0.28885799646377563), ('New Technology', 0.2793889343738556)]
[('Sport', 0.17387183010578156), ('Province', 0.1713830828666687), ('IBM', 0.16853581368923187)]
[('Country', 0.5851670503616333), ('New Product', 0.41562730073928833), ('Business Competition', 0.33475345373153687)]
[('IOT', 0.4731332063674927), ('Watson', 0.4004691243171692), ('Business Expansion', 0.3428489565849304)]
[('IBM', 0.4380996525287628), ('Partnership', 0.4356367886066437), ('Quantum Computing', 0.35531944036483765)]
[('IOT', 0.38861

In [10]:
print(len(title_classifier))

4707


In [11]:
#Output showing each topic in the taxonomy and top 10 matching articles

titles = []
for each_title in title_classifier:
    titles.append([each_title[0], each_title[1][0]])


for key, value in topic_taxonomy.items():
    print("---------")
    print(key)
    temp_label = []
    for label, keywords in value.items():
        temp_label.append(label)
    set(temp_label)
    temp_topics = []
    for each_title in titles:
        if each_title[1][0] in temp_label:
            temp_topics.append(each_title)
    sorted_list = sorted(temp_topics, key = lambda x : x[1][1], reverse = True)[:10]

    for each_title in sorted_list:
        print(feeds_read_from_file[each_title[0]])
    

---------
leader
[240, 'NAVEX Global Appoints Tech Marketing Executive Susan Beermann as Chief Marketing Officer']
[2664, 'MarTech Interview with Ani Obermeier, VP of Marketing at Smartling']
[1049, 'DXC chief executive Mike Lawrie retires']
[4372, 'Synamedia appoints Bijal Patel as CFO']
[1828, 'ACEA appoints Mr Eric-Mark Huitema as New Director General']
[4189, 'Digital Marketing Manager']
[3265, 'Triax appoints Uffe Byskov as Chief Commercial Officer']
[907, 'WestRiver Group Adds Laurel Buckner as Managing Director']
[2145, 'NFA P-TECH students learn valuable job skills at IBM']
[3879, 'Fresche Solutions Announces Appointment of New President & CEO']
---------
Tech giant
[1988, 'Friday’s Vital Data: General Electric Company (GE), International Business Machines Corp. (IBM) and Caterpillar Inc. (CAT) | InvestorPlace']
[3252, 'Tech update featuring Facebook, Google and IBM']
[3521, 'How Industrial Internet-of-Things Market is Transforming Business Overview, Applications And Demand 201